In [51]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
import Levenshtein as Lv
import json

In [2]:
data = pd.read_csv('DS_SI_KW_065_HC_OCT2OCT2022_i.csv',skiprows=2,low_memory=False)

In [3]:
data

,Account name,Campaign ID,Keyword ID,Customer ID,Ad group,Campaign,Search keyword,Search term,Search keyword match type,Campaign end date,...,Impr. (Abs. Top) %,Impr. (Top) %,Conversions,View-through conv.,Cost / conv.,Cost / conv. (Converted currency),Conv. rate,Avg. CPM,Avg. CPM (Converted currency),Conv. value
0,HealthCare.com O65,15238753096,27793000,120-348-7028,provider=social_security,O65_HC_G_ATLAS_Providers_RPA_Bucket:HumanaMST,social security,social security winter haven florida,Phrase match,"Dec 31, 2037",...,44.44%,77.78%,0.00,0,--,--,0,0.0,0.0,0.00
1,HealthCare.com O65,14492040217,87908191,120-348-7028,provider=social_security#user=disabled_people,O65_HC_G_ATLAS_Providers_Broad,disability social security,can u work on disability,Broad match,"Dec 31, 2037",...,0.00%,100.00%,0.00,0,--,--,0,0.0,0.0,0.00
2,HealthCare.com O65,15240229421,40078302,120-348-7028,provider=social_security,O65_HC_G_ATLAS_Providers_RPA_Bucket:HumanaEST,ssa,what is my social security benefit for 2022,Phrase match,"Dec 31, 2037",...,100.00%,100.00%,0.00,0,--,--,0,0.0,0.0,0.00
3,HealthCare.com O65,14492040205,139152090,120-348-7028,program=medicare,O65_HC_G_ATLAS_Generic_RPA_Bucket:High,medi care,medicare mindset,Phrase match,"Dec 31, 2037",...,0,0,0.00,0,--,--,0,0.0,0.0,0.00
4,HealthCare.com O65,15240229421,40078302,120-348-7028,provider=social_security,O65_HC_G_ATLAS_Providers_RPA_Bucket:HumanaEST,ssa,htt www ssa gov retire2 military htm,Phrase match,"Dec 31, 2037",...,0.00%,100.00%,0.00,0,--,--,0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2709045,HealthCare.com O65,12670520628,145807173,120-348-7028,1T_Disabled_General,O65_HC_SL_G_RO_General_AD_1T_EP,disability health insurance,short term disability insurance not through em...,Phrase match,"Dec 31, 2037",...,0.00%,0.00%,0.00,0,--,--,0,0.0,0.0,0.00
2709046,HealthCare.com O65,15240229424,597977466446,120-348-7028,program=medicaid#provider=selecthealth,O65_HC_G_ATLAS_Medicaid_RPA_Bucket:HumanaEST,select health medicaid,select medicaid plan,Phrase match,"Dec 31, 2037",...,0.00%,100.00%,0.00,0,--,--,0,0.0,0.0,0.00
2709047,HealthCare.com O65,17604444717,298691847025,120-348-7028,product=vision_insurance,O65_HC_G_ATLAS_Vision,vision insurance companies,affordable health and dental insurance plans,Broad match,"Dec 31, 2037",...,0,0,0.00,0,--,--,0,0.0,0.0,0.00
2709048,HealthCare.com O65,15240229415,298963608945,120-348-7028,carrier=blue_cross_blue_shield#policy_detail=p...,O65_HC_G_ATLAS_Carriers_RPA_Bucket:HumanaEST,blue cross advantage plan,bcbs of wny medicare advantage plans,Phrase match,"Dec 31, 2037",...,0.00%,100.00%,0.00,0,--,--,0,0.0,0.0,0.00


In [4]:
data.dtypes

Account name                          object
Campaign ID                            int64
Keyword ID                             int64
Customer ID                           object
Ad group                              object
Campaign                              object
Search keyword                        object
Search term                           object
Search keyword match type             object
Campaign end date                     object
Campaign start date                   object
Search terms match type               object
Currency code                         object
Clicks                                object
Impr.                                 object
CTR                                   object
Avg. CPC                              object
Converted currency code               object
Avg. CPC (Converted currency)         object
Cost                                 float64
Cost (Converted currency)            float64
Impr. (Abs. Top) %                    object
Impr. (Top

In [5]:
data['Conv_val'] = pd.to_numeric(data['Conv. value'].str.replace(',',''))

In [8]:
data['Conversions_R'] = pd.to_numeric(data['Conversions'].str.replace(',',''))

### Levenshtein score

In [9]:
data['LV_distance'] = data.apply(lambda x: Lv.distance(x['Search term'],  x['Search keyword']), axis=1)

In [10]:
data['LV_distance']

0          21
1          20
2          40
3           9
4          33
           ..
2709045    37
2709046    12
2709047    33
2709048    19
2709049    17
Name: LV_distance, Length: 2709050, dtype: int64

### Cosine Simscore

In [13]:
import math
import re
from collections import Counter
import pandas as pd

In [14]:
WORD = re.compile(r"\w+")


def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)


In [15]:
data['query_vec']=data['Search term'].apply(lambda x: text_to_vector(x)) 
data['keyword_vec']=data['Search keyword'].apply(lambda x: text_to_vector(x)) 
data['simscore']=data.apply(lambda x: get_cosine(x['query_vec'],x['keyword_vec']),axis=1)

In [16]:
data[['simscore','LV_distance','Search terms match type','Search keyword match type']]

,simscore,LV_distance,Search terms match type,Search keyword match type
0,0.632456,21,Phrase match,Phrase match
1,0.258199,20,Broad match,Broad match
2,0.000000,40,Phrase match (close variant),Phrase match
3,0.000000,9,Phrase match (close variant),Phrase match
4,0.377964,33,Phrase match,Phrase match
...,...,...,...,...
2709045,0.436436,37,Phrase match (close variant),Phrase match
2709046,0.666667,12,Phrase match (close variant),Phrase match
2709047,0.235702,33,Broad match,Broad match
2709048,0.204124,19,Phrase match (close variant),Phrase match


In [27]:
data.loc[data['Search terms match type']==data['Search keyword match type']][['Search term','Search keyword','simscore','LV_distance','Search terms match type','Search keyword match type']]


,Search term,Search keyword,simscore,LV_distance,Search terms match type,Search keyword match type
0,social security winter haven florida,social security,0.632456,21,Phrase match,Phrase match
1,can u work on disability,disability social security,0.258199,20,Broad match,Broad match
4,htt www ssa gov retire2 military htm,ssa,0.377964,33,Phrase match,Phrase match
5,social security in beachwood ohio,social security,0.632456,18,Phrase match,Phrase match
6,illinois social security card,social security,0.707107,14,Phrase match,Phrase match
...,...,...,...,...,...,...
2709041,principal health insurance,dental insurance,0.408248,13,Broad match,Broad match
2709043,bright healt,medicare health insurance,0.000000,18,Broad match,Broad match
2709044,american continental medicare supplement,continental medicare supplement,0.866025,9,Phrase match,Phrase match
2709047,affordable health and dental insurance plans,vision insurance companies,0.235702,33,Broad match,Broad match


In [28]:
data.loc[data['Search terms match type']!=data['Search keyword match type']][['Search term','Search keyword','simscore','LV_distance','Search terms match type','Search keyword match type']]

,Search term,Search keyword,simscore,LV_distance,Search terms match type,Search keyword match type
2,what is my social security benefit for 2022,ssa,0.000000,40,Phrase match (close variant),Phrase match
3,medicare mindset,medi care,0.000000,9,Phrase match (close variant),Phrase match
10,how to sign up for ss online,how to apply for social security,0.462910,19,Phrase match (close variant),Broad match
14,is everyone eligible for social security,ssa,0.000000,37,Phrase match (close variant),Phrase match
15,medicare dual advantage,medicare advantage plans,0.666667,11,Phrase match (close variant),Phrase match
...,...,...,...,...,...,...
2709038,solstice plus dental plan,dental insurance,0.353553,21,Phrase match (close variant),Phrase match
2709042,fehb vs medicare advantage,medical advantage,0.353553,10,Phrase match (close variant),Broad match
2709045,short term disability insurance not through em...,disability health insurance,0.436436,37,Phrase match (close variant),Phrase match
2709046,select medicaid plan,select health medicaid,0.666667,12,Phrase match (close variant),Phrase match


In [29]:
not_match = data.loc[data['Search terms match type']!=data['Search keyword match type']]

In [30]:
not_match

,Account name,Campaign ID,Keyword ID,Customer ID,Ad group,Campaign,Search keyword,Search term,Search keyword match type,Campaign end date,...,Conv. rate,Avg. CPM,Avg. CPM (Converted currency),Conv. value,Conv_val,Conversions_R,LV_distance,query_vec,keyword_vec,simscore
2,HealthCare.com O65,15240229421,40078302,120-348-7028,provider=social_security,O65_HC_G_ATLAS_Providers_RPA_Bucket:HumanaEST,ssa,what is my social security benefit for 2022,Phrase match,"Dec 31, 2037",...,0,0.0,0.0,0.00,0.0,0.0,40,"{'what': 1, 'is': 1, 'my': 1, 'social': 1, 'se...",{'ssa': 1},0.000000
3,HealthCare.com O65,14492040205,139152090,120-348-7028,program=medicare,O65_HC_G_ATLAS_Generic_RPA_Bucket:High,medi care,medicare mindset,Phrase match,"Dec 31, 2037",...,0,0.0,0.0,0.00,0.0,0.0,9,"{'medicare': 1, 'mindset': 1}","{'medi': 1, 'care': 1}",0.000000
10,HealthCare.com O65,14492040226,870351671,120-348-7028,provider=social_security#purchase_intent=apply...,O65_HC_G_ATLAS_Competitors_RPA_Bucket:Misc,how to apply for social security,how to sign up for ss online,Broad match,"Dec 31, 2037",...,0,0.0,0.0,0.00,0.0,0.0,19,"{'how': 1, 'to': 1, 'sign': 1, 'up': 1, 'for':...","{'how': 1, 'to': 1, 'apply': 1, 'for': 1, 'soc...",0.462910
14,HealthCare.com O65,15240229421,40078302,120-348-7028,provider=social_security,O65_HC_G_ATLAS_Providers_RPA_Bucket:HumanaEST,ssa,is everyone eligible for social security,Phrase match,"Dec 31, 2037",...,0,0.0,0.0,0.00,0.0,0.0,37,"{'is': 1, 'everyone': 1, 'eligible': 1, 'for':...",{'ssa': 1},0.000000
15,HealthCare.com O65,14492040208,1089831011,120-348-7028,policy_detail=plans#program=medicare_advantage,O65_HC_G_ATLAS_Generic_RPA_Bucket:Low,medicare advantage plans,medicare dual advantage,Phrase match,"Dec 31, 2037",...,0,0.0,0.0,0.00,0.0,0.0,11,"{'medicare': 1, 'dual': 1, 'advantage': 1}","{'medicare': 1, 'advantage': 1, 'plans': 1}",0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2709038,HealthCare.com O65,14492040211,10185031,120-348-7028,product=dental_insurance,O65_HC_G_ATLAS_Generic_RPA_Bucket:Mid,dental insurance,solstice plus dental plan,Phrase match,"Dec 31, 2037",...,0,0.0,0.0,0.00,0.0,0.0,21,"{'solstice': 1, 'plus': 1, 'dental': 1, 'plan'...","{'dental': 1, 'insurance': 1}",0.353553
2709042,HealthCare.com O65,14492040202,295727369759,120-348-7028,medicaid_provider=medi_cal#program=medicare_ad...,O65_HC_G_ATLAS_Generic_Broad,medical advantage,fehb vs medicare advantage,Broad match,"Dec 31, 2037",...,0.00%,2300.0,2300.0,0.00,0.0,0.0,10,"{'fehb': 1, 'vs': 1, 'medicare': 1, 'advantage...","{'medical': 1, 'advantage': 1}",0.353553
2709045,HealthCare.com O65,12670520628,145807173,120-348-7028,1T_Disabled_General,O65_HC_SL_G_RO_General_AD_1T_EP,disability health insurance,short term disability insurance not through em...,Phrase match,"Dec 31, 2037",...,0,0.0,0.0,0.00,0.0,0.0,37,"{'short': 1, 'term': 1, 'disability': 1, 'insu...","{'disability': 1, 'health': 1, 'insurance': 1}",0.436436
2709046,HealthCare.com O65,15240229424,597977466446,120-348-7028,program=medicaid#provider=selecthealth,O65_HC_G_ATLAS_Medicaid_RPA_Bucket:HumanaEST,select health medicaid,select medicaid plan,Phrase match,"Dec 31, 2037",...,0,0.0,0.0,0.00,0.0,0.0,12,"{'select': 1, 'medicaid': 1, 'plan': 1}","{'select': 1, 'health': 1, 'medicaid': 1}",0.666667


In [32]:
not_match['Search terms match type'].value_counts()

Phrase match (close variant)    811039
Exact match (close variant)     371637
Phrase match                      5798
Exact match                       1293
Name: Search terms match type, dtype: int64

In [31]:
not_match['Search keyword match type'].value_counts()

Phrase match    733424
Exact match     334126
Broad match     122217
Name: Search keyword match type, dtype: int64

In [38]:
not_match.loc[data['Search keyword match type']=='Broad match']['Conversions_R'].value_counts()

0.00     118894
1.00       1863
2.00        757
3.00        179
0.50        113
          ...  
5.01          1
30.00         1
18.83         1
62.50         1
28.00         1
Name: Conversions_R, Length: 102, dtype: int64

In [39]:
not_match.loc[data['Search keyword match type']=='Exact match']['Conversions_R'].value_counts()

0.00     321942
1.00       6932
2.00       2839
3.00        683
0.50        356
          ...  
11.97         1
16.83         1
34.84         1
3.58          1
36.00         1
Name: Conversions_R, Length: 232, dtype: int64

In [40]:
not_match.loc[data['Search keyword match type']=='Phrase match']['Conversions_R'].value_counts()

0.00     717315
1.00       9506
2.00       3754
3.00        864
0.50        369
          ...  
22.88         1
97.00         1
33.83         1
3.42          1
17.67         1
Name: Conversions_R, Length: 221, dtype: int64

In [54]:
not_match.loc[data['Search terms match type']=='Phrase match (close variant)']['Conversions_R'].value_counts()

0.00     792941
1.00      10640
2.00       4194
3.00        965
0.50        449
          ...  
14.50         1
32.17         1
69.67         1
8.82          1
17.67         1
Name: Conversions_R, Length: 239, dtype: int64

In [55]:
not_match.loc[data['Search terms match type']=='Exact match (close variant)']['Conversions_R'].value_counts()

0.00     358289
1.00       7562
2.00       3119
3.00        756
0.50        383
          ...  
16.83         1
34.84         1
35.08         1
16.17         1
36.00         1
Name: Conversions_R, Length: 253, dtype: int64

In [57]:
not_match.loc[data['Search terms match type']=='Phrase match']['Conversions_R'].value_counts()

0.00     5712
1.00       56
2.00       18
0.50        3
3.00        2
0.01        1
0.83        1
7.99        1
12.50       1
0.60        1
1.50        1
4.00        1
Name: Conversions_R, dtype: int64

In [58]:
not_match.loc[data['Search terms match type']=='Exact match']['Conversions_R'].value_counts()

0.00     1209
1.00       43
2.00       19
4.00        3
0.50        3
3.00        3
5.00        1
0.58        1
10.01       1
18.00       1
22.00       1
7.00        1
10.00       1
6.00        1
6.50        1
1.50        1
0.01        1
4.50        1
3.17        1
Name: Conversions_R, dtype: int64

In [42]:
data['Search keyword'].value_counts()

social security                             70122
medicare                                    47546
dental insurance                            39679
healthcare com                              37814
medical medicare                            36177
                                            ...  
medicare 2021 cost                              1
difference between medicare part a and b        1
does medicare part b cost in 2021               1
what does medicare and part b cover             1
medicare advantage plans ppo vs hmo             1
Name: Search keyword, Length: 10067, dtype: int64

In [52]:
json.loads(data['Search term'].value_counts().head(100).to_json())

{'how do i sign up for medicare part b if i already have part a': 655,
 'i am 64 when should i apply for medicare': 484,
 'when should i sign up for medicare part b if i am still working': 477,
 'medicare part b deductible 2022': 461,
 'medicare part b premium 2022 chart': 442,
 'difference between medicare part a and medicare part b': 435,
 'how long does it take to get medicare part b after applying': 427,
 'medicare give back program by zip code': 414,
 'what will medicare cost in 2022': 413,
 'medicare premiums 2022': 403,
 'www socialsecurity gov medicare apply html': 397,
 'social security medicare': 389,
 'medicare part a b c d explained': 384,
 'social security medicare part b': 375,
 'can i sign up for medicare part b online': 375,
 'medicare part b form': 373,
 'social security medicare premiums 2022': 360,
 'can i get medicare part b for free': 359,
 'medicare advantage vs medicare supplement': 357,
 'medicare part a and b coverage chart': 350,
 'what is the cost of medicare